In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import os 
import missingno as msno
from plotly import express as px

from data_cleanup import *
from feature_selection import *
from model_ import *
from country_evaluation import *

from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as rmse
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, LassoLarsCV
from sklearn.ensemble import RandomForestRegressor

## Questions
Which country’s characteristics are good predictors of a country’s position on a corruption index?

If we manage to find characteristics and predict country’s position on a corruption index reasonably well, are there countries whose characteristics don’t correspond to their position on a corruption index?

If yes, does this hold across all corruption indexes, or do the lists of such countries vary from index to index?

We chose these questions because corruption indices are very rough estimates based on perception and trying to replicate the data with country characteristics seemed interesting and worth doing. Corruption indices often get criticized about potential biases so we also wanted to check if we would find some patterns regarding outliers in our data.

## Dataset Loading
Our main dataset is the [QoG (Quality of Governence) Standard Time-Series Dataset](https://www.gu.se/en/quality-government/qog-data/data-downloads/standard-dataset) from the University of Gothenburg. It is a huge dataset about the quality of governance with 2000 features that includes data from 1946 to 2021 for most countries. We chose it because we never really had to deal with dimensionality reduction yet and where interested in doing so. We additionally merge this dataset with the [ISO-3166-Countries with Regions](https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv) dataset to additionally obtain sub-region (f.e. Western Europe, South-East Asia, etc.) information, as we think the subregion of a country is very important for its corruption rankings. Due to this we are using the subregion to split the data in a stratified fashion into training/test-data.

For region merging manual patching was required as some countries that where included in the QoG dataset where not inside the ISO-3166 dataset. Additionally we opted to merge the Micronesia/Melanesia/Polynesia subregions into a new, combined region called 'Pacific Island' as the original subregions only had a small amount of entries.

In [ ]:
data_dir = 'data'
qog_dataset_filename = 'qog_std_ts_jan22.csv'
df = pd.read_csv(join(data_dir, qog_dataset_filename), low_memory=False)

df = merge_region(df)
display(df)


## Dataset Exploration

We can see that there are some features with a very small amount of non-null values. We can also see that most features are numerical values or might be encoded as numerical values. Checking this with the documentation of the dataset shows that this indeed seems to be the case as all categorical variables were encoded numerically.

In [ ]:
df.info(verbose=True, memory_usage='deep', show_counts=True)
df.describe()
df.dtypes.value_counts()

Next we are looking at the correlation between corruption indices that we decided to investigate more closely after our preliminary research.
We can see that most of them are highly correlated which makes sense as they generally try to estimate the same thing with slightly different approaches.

In [ ]:
corruption_col = ['bci_bci', 'ti_cpi', 'vdem_corr', 'vdem_execorr', 'vdem_jucorrdc', 'vdem_pubcorr', 'wbgi_cce', 'ti_cpi_om']
corruption_corr = df[corruption_col].corr()
mask = np.triu(np.ones_like(corruption_corr, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corruption_corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

As numbers don't tell the whole story we also look at pairplots between the different corruption indices. The main corruption indices (bci_bci, ti_cpi and wbgi_cce) seem to be really close in their estimates.

In [ ]:
sns.pairplot(df[corruption_col])

We are looking at missing values next. We can see that previously less data was available. The closer we are to recent times the more data was available. We can also see a somewhat regular-ish pattern of missing values across multiple years. This most likely are countries for which no data was gathered across multiple years (f.e. Western countries generally started collecting extensive data earlier while countries in sub-saharan africa where rather late). 

Due to changes in the methodology for ti_cpi (which we can see quite nicely in the plots) we have to handle this case in a more special case. Explanations for it follow in the next steps.

As we have 2000 features it doesn't really make sense to look at the distribution/value-ranges/missing values, ... etc for those, as this would take an unreasonable amount of time to do.

In [ ]:
# drop every row where none of the corruption data is available
df_any_corruption_info_available = df.dropna(subset=corruption_col, axis=0, how="all")

# drop every row where not every corruption data is available with special handling for ti_cpi
df_cpi_combined = df.copy()
df_cpi_combined['ti_cpi']=df['ti_cpi'].combine_first(df['ti_cpi_om'])
df_all_corruption_info_available = df_cpi_combined.dropna(subset=corruption_col, axis=0, how='any')


display(df_any_corruption_info_available.shape)
#11k to 1.7k rows
display(df_all_corruption_info_available.shape)

corruption_col_with_year = ['year', 'bci_bci', 'ti_cpi', 'ti_cpi_om', 'vdem_corr', 'vdem_execorr', 'vdem_jucorrdc', 'vdem_pubcorr', 'wbgi_cce']
ax = msno.matrix(df[corruption_col_with_year].sort_values(by='year'))
ax.set_title('Missing values from whole dataframe sorted by year (most recent being lowest)', fontsize=18)
ax = msno.matrix(df_any_corruption_info_available[corruption_col_with_year].sort_values(by='year'))
ax.set_title('Missing values from dataframe with all rows where any corruption info is available sorted by year (most recent being lowest)', fontsize=18)

corruption_col_with_year.remove('ti_cpi_om')
# No surprises here
ax = msno.matrix(df_all_corruption_info_available[corruption_col_with_year].sort_values(by='year'))
ax.set_title('Missing values from dataframe with all rows where all corruption info is available sorted by year (most recent being lowest)', fontsize=18)

Its also important to get an idea of the value range for each corruption metric.

In [ ]:
df[corruption_col].describe()

## Dataset preprocessing

In the next step we are dropping some features that were directly used by the corruption indices that we want to predict. If we would keep them our feature selection just would end up picking those features and we would only learn the exact same model that the institutions used to calculate their corruption indices. We opted for a smaller amount of corruption indices so that the scope of this exercise didn't get to big.

In [ ]:
# columns with corruption indices
corr_cols = ['ti_cpi', 'bci_bci', 'ti_cpi_om', 'wbgi_cce'] 
# columns with metadata
meta_cols = ['ccode', 'ccode_qog', 'ccodealp', 'ccodealp_year', 'ccodecow', 'cname', 'cname_qog', 'cname_year', 'version', 'year', 'region', 'sub-region']
df_reduced = drop_certain_columns(df, corr_cols, meta_cols)
display(df_reduced.shape)

The dataset contains data from 1960-2021. As most corruption indices only started to be available in the 1990s the data from the years before that are not that interesting(we only consider 'this' years data to calculate 'this' years result). Additionally we will also end up training one model for each corruption index and as we want to make comparisons between them we will have to use the exact same train and testset for all of them. Due to this we only keep the rows where values for all corruption indices are available. Unfortunately ti_cpi (Transparency international - Corruption Perception Index) drastically changed its methodology in 2012, which means that we have two variables for it ti_cpi and ti_cpi_om (old methodology), as they can't be directly compared. If we would restrict the data to only include the new methodology we would have a very small dataset. But the old methodology is not as interesting as it is a relative measure (relative to other countries in that year) that can't be directly compared across different years (so having the same ti_cpi_om-value in 1990 and 1995 could mean different things). 

Because of this we opted to keep rows where the union of ti_cpi and ti_cpi_om with the caveat that 
- those models can't be directly compared to the other models (bci_bci and wbgi_cce) because they don't have the same training/testdata
- the model for ti_cpi_om might not be that meaningful due to being a relative measure

In [ ]:
df_reduced = return_rows_where_all_corruption_data_is_available(df_reduced, corr_cols)
display(df_reduced.shape)

Next we drop all rows that have less than 10% non-nan values as they don't hold that much information anyway and because we are still left with a very large number of features.
Then we transform features that have less than 10 unique values in 2836 rows into categorical variables. This picks up a small amount of false positives but in a huge majority of cases this transformation was correct. Due to the high amount of features we didn't bother with doing it 'correctly' (which would mean by hand).

As we still have a very high amount of features we drop all columns that have any nan value so that we don't have to correctly handle them. We are left with 1210 features.

In [ ]:
df_reduced = drop_columns_with_nan_values(df_reduced)
display(df_reduced.shape)
df_reduced = transform_to_categorical(df_reduced)
display(df_reduced.shape)

df_reduced.loc[:,['ti_cpi', 'ti_cpi_om']] = df_reduced.loc[:,['ti_cpi', 'ti_cpi_om']].replace(np.NaN, -5)
df_reduced = df_reduced.dropna(how='any', axis=1).copy()
df_reduced.loc[:,['ti_cpi', 'ti_cpi_om']] = df_reduced.loc[:,['ti_cpi', 'ti_cpi_om']].replace(-5, np.NaN)
display(df_reduced.shape)
df = df_reduced

### Feature selection

Due to having >1200 features we choose the best 30 according to sklearns KBest algorithm. This still leaves us with the potential problem of multicollinear features. To solve that we look at pairs of highly correlated features (>.85) and remove the one less correlated to the target variable.
As previously mentioned we use the merged subregion to split the data into a stratified test and train set. As we use the same seed we get the same training/test split for bci and wbgi. 

In [ ]:
df = drop_date_columns(df)

best_features_dict = {}
selected_features_dict = {}

for target_col in corr_cols:
    X_train, X_test, y_train, y_test = create_traintestsplit(df, corr_cols = corr_cols, meta_cols=meta_cols, target_col=target_col)
    
    best_features = pre_select(X_train, y_train)
    best_features_dict[target_col] = set(best_features)
    df_train = X_train[best_features].copy()
    df_train[target_col]=y_train
    mce = MultiCollinearityEliminator(df_train, target_col, 0.85)
    feaures_no_collinearity = list(mce.autoEliminateMulticollinearity().columns)
    feaures_no_collinearity.remove(target_col)
    selected_features_dict[target_col] = set(feaures_no_collinearity)

## Modeling

To model the data we tried out a Lasso linear model and a random forest regressor. Based on our pre selection process we train our models on different feature configurations:

    - the individual selected features for a particular index
    - the intersection of all selected features for all indices
    - the union of selected features for all indices
    - the union of selected features for all indices without filtering based on multicollinearity

To report the accuracy of the model we used the proven r2 and the rmse as metrics.
    
### Lasso
The used library uses cross validation to determine a good value for alpha.

In [ ]:
selected_features_union=list(set.union(*list(selected_features_dict.values())))
selected_features_intersection=list(set.intersection(*list(selected_features_dict.values())))

In [ ]:
best_features_union=list(set.union(*list(best_features_dict.values())))
best_features_intersection=list(set.intersection(*list(best_features_dict.values())))

In [ ]:
def lasso_info_script(features, name):
    lasso_bf = dict()

    df_score = pd.DataFrame(columns=['r2', 'rmse'], index=corr_cols)
    for target in corr_cols:
        if isinstance(features, dict):
            lasso_bf[target] = apply_lassocv(df, target, list(features[target]), corr_cols, meta_cols, fprint=False)
        else:
            lasso_bf[target] = apply_lassocv(df, target, features, corr_cols, meta_cols, fprint=False)
        df_score.loc[target,] = [lasso_bf[target]['r2'] ,lasso_bf[target]['rmse']]
    
    print('scores:' + name)
    display(df_score)
    
lasso_info_script(selected_features_dict, ' Selected Features')
lasso_info_script(selected_features_intersection, 'Features intersection')
lasso_info_script(selected_features_union, 'selected Features union')
lasso_info_script(best_features_union, ' selected Features union without collinearity filter')

# print(len(best_features_union))
# print(len(selected_features_intersection))

We can see here that the models perform all very similar with a slight edge for the models with more features. This was to be expected, due to the fact, that the features where preselected. It can also be seen, that the the multicollinearity does not seem to pose a problem to the model since the unfiltered features do not lead to worse predictions.

### Random Forest

Next we do the same for a Random Forest Regressor. Here initially no cross validation is done. We just use a default setup at first.

In [ ]:
def rf_info_script(features, name):
    rf_bf = dict()

    df_score = pd.DataFrame(columns=['r2', 'rmse'], index=corr_cols)
    for target in corr_cols:
        if isinstance(features, dict):
            rf_bf[target] = apply_rf(df, target, list(features[target]), corr_cols, meta_cols, fprint=False)
        else:
            rf_bf[target] = apply_rf(df, target, features, corr_cols, meta_cols, fprint=False)
        df_score.loc[target,] = [rf_bf[target]['r2'] ,rf_bf[target]['rmse']]
    
    print('scores:' + name)
    display(df_score)

rf_info_script(selected_features_dict, ' Selected Features')
rf_info_script(selected_features_intersection, 'Features intersection')
rf_info_script(selected_features_union, 'selected Features union')
rf_info_script(best_features_union, ' selected Features union without collinearity filter')

Random Forest performs very  similarly to the lasso model allthough no parameter optimization is done yet. So we continue with Random Forrest and do hyperparameter optimization for some specific settings next to further optimize the results.

### Grid Search: Random Forest

With cross validation / hyperparameter grid search better parameters are determined. With those optimizations then again models are trained, then the test set is predicted and scores are evaluated. We where not able to set a random state for this part, to accommodate this fact we saved the results via pickle so we can work on a consistent interpretation of the data.

In [ ]:
def rf_gridsearch_info_script(features, name):
    rf_bf = dict()

    param_grid = {
        "randomforestregressor__max_depth": [2, 3, 5, 10, None],
        "randomforestregressor__min_samples_split": [2, 3, 5, 10],
        "randomforestregressor__max_features": ["log2", None]
        }

    df_score = pd.DataFrame(columns=['r2', 'rmse'], index=corr_cols)
    for target in corr_cols:
        if isinstance(features, dict):
            rf_bf[target] = apply_gridsearch_rf(df, target, list(features[target]), param_grid, corr_cols, meta_cols, fprint=False)
        else:
            rf_bf[target] = apply_gridsearch_rf(df, target, features, param_grid, corr_cols, meta_cols, fprint=False)
        df_score.loc[target,] = [rf_bf[target]['r2'] ,rf_bf[target]['rmse']]
    
    print('score: Grit Search Random Forrest selected Features union')
    display(df_score)

    # file = os.path.join('pickle', name +'.obj')
    # f = open(file, 'wb')
    # pickle.dump(rf_bf ,f)
    #f.close()    

rf_gridsearch_info_script(selected_features_union, 'selected Features union')

In [ ]:

f = open('pickle/rf_grid_selected_features_union.obj', 'rb')
rf_bf = pickle.load(f)
f.close()

df_score = pd.DataFrame(columns=['r2', 'rmse'], index=corr_cols)
for target in corr_cols:
    df_score.loc[target,] = [rf_bf[target]['r2'] ,rf_bf[target]['rmse']]

print('score: Grit Search Random Forrest selected Features union')
display(df_score)



Our best scoring model is the 'selected Features union' one, we therefore select it for all further analysis.  In the scores table we can see, that after hyperparameter optimization the metrics of our model improved again a bit. R2 of above 0.7 has been reached for 3 out of the four corruption indices and the rmse values seem reasonable as well in regards to the scale our corruption indices operate on. 
- ti_cpi: 0-100
- bci_bci: 0-100
- ti_cpm: 0-10
- wbgi_cce: -2.5-2.5

In [ ]:
l_fi = [rf_bf[target]['feat_importance'] for target in corr_cols]
df_fi = pd.concat(l_fi)

df_fi = df_fi.reindex(df_fi.mean().sort_values(ascending=False).index, axis=1)
col_names = df_fi.columns
df_fi = df_fi.T.melt(
    ignore_index=False,
    value_vars = ['ti_cpi', 'bci_bci', 'ti_cpi_om', 'wbgi_cce'],
    value_name = 'feature_importance'
).reset_index().rename(columns={'index': 'feature', 'variable': 'corruption_index'})

plt.rcdefaults()
font = {'family' : 'normal',
    'size'   : 14}

plt.rc('font', **font)
plt.figure(figsize=(20,8))
sns.barplot(df_fi, x='feature',  y='feature_importance', hue='corruption_index', palette='magma', width=0.6)
plt.xticks(rotation=90)
plt.legend(loc='upper right')
plt.grid()
plt.title('Feature importance')

Looking at the feature importance in the plot above we can see that the features have comparable levels of relevance to the different corruption indexes.
The plot shows that our model sees  political and civil freedom aswell as live expectancy as good indicators for corruption levels this trend continues in the less important features aswell. This makes sense because it aligns with the general consensus that free, democratic countries suffer less from the evil that is corruption. A full list of the most important features can be seen below.

| code        | Description                 |
| ----------------- | --------------------------------------------------------------------------------------------------------------|
| fh_pr_1.0         | Political Rights Rating (1 = most free)          |
| fh_ipolity2       | Level of Democracy                               |
| ihme_lifexp_0104t | Life Expectancy, Both sexes, Age 1-4 years       |
| fh_cl_1.0         | Civil Liberties (1 = most free)                              |
| ihme_lifexp_0104m | Life Expectancy, Male, Age 1-4 years             |
| br_mon            | Is the country a monarchy                        |
| cpds_vper_0.0     | Share of votes: personalist                      |
| gd_ptss_1.0       | Political Terror Scale - US State Department (1. Countries under a secure rule of law, people are not imprisoned for their view, and torture is rare or exceptional. Political murders are extremely rare.)    |
| kun_cluster_5.0   | Cluster memberships based on means (5 = mostly western)              |
| wel_sys_1.0       | Political System Type (1=Unbound Autocracy)                           |
| cpds_lall_0.0     | Share of seats in parliament: electoral alliance |
| fh_status_1.0     | Freedom Status (1=Free)                                |
| ciri_injud_2.0    | Independence of the Judiciary  (2=generally independent)                  |
| fhp_status5_1.0   | Freedom of the Press, Status (2001-2016)  (1 = Free )       |
| wel_scalezone_4.0 | Scalezone on Citizen Rights  (4=completely democratic)                    |
| cpds_chg_0.0      | Number of changes in government per year         |

### Evaluation

In [ ]:
relative_errs = get_rel_err_df(rf_bf, df, corr_cols)

#### By region
There is no big difference between performance of models across regions. ti_cpi models have slightly more problems with predicting Oceania. Oceania has smaller range or relative errors because it contains only 1 country: Papua New Guinea.
We can also see that wbgi_cce has much larger number of outliers in comparison to other models.

In [ ]:
get_plots_by_region(relative_errs)

#### By sub-region
##### Africa
We can see that ti_cpi model performs much worse for Northern Africa than for Sub-Saharan Africa, whereas for wbgi_cce it's the opposite. The countries with the very big outliers all belong to the Sub-Saharan Africa, but they are not the same across different indices: ti_cpi and ti_cpi_om models have problems when predicting values for Somalia, but bci_bci model has problems with Botswana and wbgi_cce model has problems with predicting values for Ghana and South Africa. These problems persist over multiple years for each country, but not across indices, hence with high probability the problem lies in the prediction models and not in the mismatch between countrie's characteristics and position on the index.
##### Europe
Big relative error range shows that all models have more difficulties predicting values for countries from Eastern Europe. For bci_bci, the relative error for Eastern Europe is significantly worse than for other European regions. Since this trend persist over all the indeces, one could say that there is a suspicious mismatch between predictors and the estimated corruption index.

For other regions different models perform slightly differently (i.e., ti_cpi and ti_cpi_om models have more problems with Southern Europe, whereas bci_bci model has difficulties with Northern Europe), but since they don't hold across the indices they are less interesting for us.

Interestingly multiple models (ti_cpi_om, wgbi_cce) have Latvia from the beginning of the 2000s as an outlier, but the years are different. This could point to some bias in the corruption metrics, but would need further investigation.
##### Asia
Different models have slightly different performance for countries of each region of Asia, but all in all there are no major differences in the performance of the models and interesting common connections which would hold across the indices.
##### Oceania
As were said previously, Oceania contains only 1 country (Papua New Guinea) and only 1 sub-region (Pacific Islands) in the test set, so it is not possible to make any comparisons.
##### Americas
Since North America is extremely small and contains only 2 countries in total, it is not represented in the test dataset.

From the big range of relative errors we can see that bci_bci model has much more difficulties predicting values for countries from Latin America and the Caribbean than countries from other sub-regions. For other models the results are comparable. The performance of other models are comparable for this region. The outlier countries are same within models, but differ across different models, so there are also no interesting connections here.


In [ ]:
get_plots_by_subregion(relative_errs)

### Difficulties and learnings

Handling a time series Dataset of a size like this brought some unique challenges that we haven't faced before. Due to the sheer size manual processing of the singular features was impossible. This meant that we had to use tools like sklearns KBest for feature reduction to get to a reasonable starting point for our models. Another big problem was to identify categorical variables since this information is not encoded into the dataset and manually adding this information with over 2000 features was not feasible. Because of this we automatically transform every variable with less then 11 unique values to one hot encoded variables. This of course leads to some ordinal variables being transformed to categorical which can also be seen in the most important features. Excluding those variables from the transformation changes the importance which leads to other features taking their place which often repeats the cycle. Even though we have over 2000 samples the number of countries per region in our test set is quite limited. This is  due to the fact that each sample belongs to a country and a year, and we have to split test and train sets in a way that all data points of one country are either in the test or the train set. We do this to ensure that our test data is unseen because the differences in between years might be too small. Besides the difficulties we improved our knowledge of ```pandas```, ```seaborn```, ```sklearn``` and also learned completly new libraries like ```plotly``` and ```msno```

### Biases

One bias might be the small sample size of certain regions, which leads to under-representation in the model and therefore higher uncertainty which could lead to an inaccurate prediction. But the bigger issues stem from the data itself. All our target values and most features are collected by western institutions which could lead to some biases due to imposing western values and standards onto the non western part of the world. Apart from that western institutions might unintentionally tend to western favoring metrics.

### Work split

Every teammate was involved in every step, but there were still some main focus points:

Godun Alina: Feature Selection, Evaluation
Habeger David: Data Exploration, Data preprocessing
Haberl Alexander: Data Exploration, Data preprocessing
Konzett Siemon: Modeling